In [310]:
import pandas as pd
import numpy
train = pd.read_csv("copied_to_txt.txt", delimiter="\t", header=0)
from bs4 import BeautifulSoup
import re
import nltk


In [312]:
training = []
category=[]
subcategory=[]
sentence=[]
with open('copied_to_txt.txt', 'r') as fingood:
     for review in fingood.readlines():
            cat = 0
            subcat = 0
            for i,c in enumerate(review):
                if(review[i] == ':'and cat == 0):
                    cat = i
                if(review[i] == " " and subcat == 0):
                    subcat = i
           
            category.append(review[:cat])
            subcategory.append(review[cat+1:subcat])
            sentence.append(review[subcat+1:])
            
training = pd.DataFrame( data={"category":category, "subcategory":subcategory, "sentence":sentence} )
print (training)
                    
            

     category                                           sentence subcategory
0     ï»¿DESC  How did serfdom develop in and then leave Russ...      manner
1        ENTY  What films featured the character Popeye Doyle...      cremat
2        DESC  How can I find a list of celebrities ' real na...      manner
3        ENTY  What fowl grabs the spotlight after the Chines...      animal
4        ABBR                  What is the full form of .com ?\n         exp
5         HUM  What contemptible scoundrel stole the cork fro...         ind
6         HUM  What team did baseball 's St. Louis Browns bec...          gr
7         HUM                  What is the oldest profession ?\n       title
8        DESC                         What are liver enzymes ?\n         def
9         HUM  Name the scar-faced bounty hunter of The Old W...         ind
10        NUM                    When was Ozzy Osbourne born ?\n        date
11       DESC  Why do heavier objects travel downhill faster ?\n      reason

In [313]:
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
words = []
cat = []
subcat=[]
documents = []
ignore_words = ['\n']
numrange = training['sentence'].size

for i in range( 1, numrange ):
    w = nltk.word_tokenize(training['sentence'][i])
    words.extend(w)
    #documents.append((w, training['category'][i]))
    #documents.append((w, training['subcategory'][i]))
    if training['category'][i] not in cat:
        cat.append(training['category'][i])
    if training['subcategory'][i] not in subcat:
        subcat.append(training['subcategory'][i])

# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = list(set(words))

# remove duplicates
cat = list(set(cat))

print (len(documents), "documents")
print (len(cat), "category",cat)
print (len(subcat), "subcategory",subcat)
print (len(words), "unique stemmed words")


0 documents
6 category ['LOC', 'ENTY', 'NUM', 'ABBR', 'HUM', 'DESC']
47 subcategory ['cremat', 'manner', 'animal', 'exp', 'ind', 'gr', 'title', 'def', 'date', 'reason', 'event', 'state', 'desc', 'count', 'other', 'letter', 'religion', 'food', 'country', 'color', 'termeq', 'city', 'body', 'dismed', 'mount', 'money', 'product', 'period', 'substance', 'sport', 'plant', 'techmeth', 'volsize', 'instru', 'abb', 'speed', 'word', 'lang', 'perc', 'code', 'dist', 'temp', 'symbol', 'ord', 'veh', 'weight', 'currency']
6409 unique stemmed words


In [314]:
def review_to_words( raw_review ):
   
    # 1. Remove HTML
    review_text = BeautifulSoup(raw_review, "lxml").get_text() 
    #
    # 2. Convert to lower case, split into individual words
    words = review_text.lower().split()                             
   
    # 3. Join the words back and return the result.
    return( " ".join( words )) 

In [315]:
num_reviews = training["sentence"].size
output_cat = []
output_subcat = []
output_empty_cat = [0] * len(cat)
output_empty_subcat = [0] * len(subcat)

# Initialize an empty list to hold the clean reviews
clean_train_reviews = []

for i in range( 1, num_reviews ):
    
    output_row_cat = list(output_empty_cat)
    output_row_cat[cat.index(training["category"][i])] = 1
    output_cat.append(output_row_cat)
    output_row_subcat = list(output_empty_subcat)
    output_row_subcat[subcat.index(training["subcategory"][i])] = 1
    output_subcat.append(output_row_subcat)

    clean_train_reviews.append( review_to_words( training["sentence"][i] ))
    

In [316]:
from sklearn.feature_extraction.text import CountVectorizer
#BAg of words of 5000 features
vectorizer = CountVectorizer(analyzer = "word", tokenizer = None, preprocessor = None,stop_words = None, max_features = 5000)
train_data_features = vectorizer.fit_transform(clean_train_reviews)
train_data_features = train_data_features.toarray()
print (train_data_features.shape)

(5451, 5000)


In [317]:
print (output_cat[10])

[0, 0, 0, 0, 0, 1]


In [318]:
print (output_subcat[10])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [319]:
import numpy as np

# compute sigmoid nonlinearity
def sigmoid(x):
    output = 1/(1+np.exp(-x))
    return output

# convert output of sigmoid function to its derivative
def sigmoid_output_to_derivative(output):
    return output*(1-output)
 

In [320]:
#Training neural network for Category part
hidden_neurons = 400
np.random.seed(8)
theta1 = 2*np.random.random((len(X[0]), hidden_neurons)) - 1
theta2 = 2*np.random.random((hidden_neurons, len(cat))) - 1
theta1.size
lamda = 3



X = (train_data_features)
y = (output_cat)
epochs = 1000
alpha = .003
dell_theta1 = np.zeros_like(theta1)
dell_theta2 = np.zeros_like(theta2)
last_mean_error = 1
start_X = X[1:1500]
final_Y = y[1:1500]
for j in iter(range(epochs+1)):
    layer_2 = sigmoid(np.dot(start_X, theta1))
    layer_3 = sigmoid(np.dot(layer_2, theta2))
    layer_3_error =   final_Y - layer_3
    if (j% 10) == 0 and j > 5:
         # if this 10k iteration's error is greater than the last iteration, break out
        if np.mean(np.abs(layer_3_error)) < last_mean_error:
            print ("delta after "+str(j)+" iterations:" + str(np.mean(np.abs(layer_3_error))) )
            last_mean_error = np.mean(np.abs(layer_3_error))
        else:
            print ("break:", np.mean(np.abs(layer_3_error)), ">", last_mean_error )
            break

    layer_2_error = (layer_3_error.dot(theta2.T))*sigmoid_output_to_derivative(layer_2)
    #layer_3_delta_sigma = layer_3_error * sigmoid_output_to_derivative(layer_2)
    #layer_2_error = layer_3_delta_sigma.dot(theta2.T)
    #layer_2_delta_sigma = layer_2_error * sigmoid_output_to_derivative(layer_2)
        
    dell_theta2 = (layer_2.T.dot(layer_3_error))
    dell_theta2 += lamda*theta2
    dell_theta1 = (start_X.T.dot(layer_2_error))
    dell_theta1 += lamda*theta1
    theta1 += alpha * dell_theta1
    theta2 += alpha * dell_theta2
    

delta after 10 iterations:0.224787134933
delta after 20 iterations:0.208270172093
delta after 30 iterations:0.179465774059
delta after 40 iterations:0.148365536693
delta after 50 iterations:0.126100320479
delta after 60 iterations:0.114380013536
delta after 70 iterations:0.1051736809
delta after 80 iterations:0.0971312561341
delta after 90 iterations:0.0899750933139
delta after 100 iterations:0.0837035012928
delta after 110 iterations:0.0781187109646
delta after 120 iterations:0.0732648974114
delta after 130 iterations:0.0690619673481
delta after 140 iterations:0.0653209456086
delta after 150 iterations:0.0620947956334
delta after 160 iterations:0.0593276165797
delta after 170 iterations:0.0569157609741
delta after 180 iterations:0.054777361297
delta after 190 iterations:0.0528523561749
delta after 200 iterations:0.0511013007598
delta after 210 iterations:0.0494885931415
delta after 220 iterations:0.0479848953192
delta after 230 iterations:0.0465904862787
delta after 240 iterations:0.0

C:\Users\anish\Anaconda3\lib\site-packages\ipykernel\__main__.py:5: RuntimeWarning: overflow encountered in exp


delta after 290 iterations:0.0408422174799
delta after 300 iterations:0.0403088818398
delta after 310 iterations:0.0398658130998
delta after 320 iterations:0.0394865660814
delta after 330 iterations:0.0391500974728
delta after 340 iterations:0.0388697874149
delta after 350 iterations:0.0386358149554
delta after 360 iterations:0.0383400077859
delta after 370 iterations:0.0380613105602
delta after 380 iterations:0.0378358997098
delta after 390 iterations:0.0375993858461
delta after 400 iterations:0.037496846851
delta after 410 iterations:0.0374406593611
delta after 420 iterations:0.0373900231134
delta after 430 iterations:0.0373289999834
delta after 440 iterations:0.0372224770653
delta after 450 iterations:0.0370802487295
delta after 460 iterations:0.0370695514604
delta after 470 iterations:0.0370364815588
delta after 480 iterations:0.0370202646162
delta after 490 iterations:0.0369850289603
break: 0.0370426851619 > 0.0369850289603


In [322]:
#Result Check
#print (theta1.size)
n=4357
l0 = X[n]
print (training['sentence'][n+1])
print (training['category'][n+1])
print (len(cat), "category",cat)
# matrix multiplication of input and hidden layer
l1 = sigmoid(np.dot(l0, theta1))
# output layer
l2 = sigmoid(np.dot(l1, theta2))

print (l2)

Who portrayed Prewett in From Here to Eternity ?

HUM
6 category ['LOC', 'ENTY', 'NUM', 'ABBR', 'HUM', 'DESC']
[ 0.  0.  0.  0.  1.  0.]


C:\Users\anish\Anaconda3\lib\site-packages\ipykernel\__main__.py:5: RuntimeWarning: overflow encountered in exp


In [323]:
#cross-validation for categories
sum  = [0,0,0,0,0,0]
for i in range( 4000, 4500 ):
    l0 = X[i]
    l1 = sigmoid(np.dot(l0, theta1))
    l2 = sigmoid(np.dot(l1, theta2))
    output_cat = np.asarray(output_cat)
    #print (l2)
    #print (output_cat[i])
    sum = sum + (l2 - output_cat[i])
    
print (np.mean(np.abs(sum)))

C:\Users\anish\Anaconda3\lib\site-packages\ipykernel\__main__.py:5: RuntimeWarning: overflow encountered in exp


5.0973215213


In [325]:
#Training neural network for SubCategory part
shidden_neurons =400
np.random.seed(5)
stheta1 = 2*np.random.random((len(X[0]), shidden_neurons)) - 1
stheta2 = 2*np.random.random((shidden_neurons, len(subcat))) - 1
#theta1.size
slamda = 8

#def train(X, y, hidden_neurons=400, alpha=0.0045, epochs=1000):


sX = (train_data_features)
sy = (output_subcat)
sepochs = 1000
salpha = .001
sdell_theta1 = np.zeros_like(stheta1)
sdell_theta2 = np.zeros_like(stheta2)
slast_mean_error = 1
start_X = sX[1:1500]
sfinal_Y = sy[1:1500]
for j in iter(range(sepochs+1)):
    slayer_2 = sigmoid(np.dot(start_X, stheta1))
    slayer_3 = sigmoid(np.dot(slayer_2, stheta2))
    slayer_3_error =   sfinal_Y - slayer_3
    if (j% 10) == 0 and j > 5:
         # if this 10k iteration's error is greater than the last iteration, break out
        if np.mean(np.abs(slayer_3_error)) < slast_mean_error:
            print ("delta after "+str(j)+" iterations:" + str(np.mean(np.abs(slayer_3_error))) )
            slast_mean_error = np.mean(np.abs(slayer_3_error))
        else:
            print ("break:", np.mean(np.abs(slayer_3_error)), ">", slast_mean_error )
            break

    slayer_2_error = (slayer_3_error.dot(stheta2.T))*sigmoid_output_to_derivative(slayer_2)
    #layer_3_delta_sigma = layer_3_error * sigmoid_output_to_derivative(layer_2)
    #layer_2_error = layer_3_delta_sigma.dot(theta2.T)
    #layer_2_delta_sigma = layer_2_error * sigmoid_output_to_derivative(layer_2)
        
    sdell_theta2 = (slayer_2.T.dot(slayer_3_error))
    sdell_theta2 += slamda*stheta2
    sdell_theta1 = (start_X.T.dot(slayer_2_error))
    sdell_theta1 += slamda*stheta1
    stheta1 += salpha * sdell_theta1
    stheta2 += salpha * sdell_theta2


delta after 10 iterations:0.025563394548
delta after 20 iterations:0.0233061564591
delta after 30 iterations:0.0220858033984
delta after 40 iterations:0.0215798781087
delta after 50 iterations:0.0212764833332
delta after 60 iterations:0.0205853841501
delta after 70 iterations:0.0196498567667
delta after 80 iterations:0.0187849511121
delta after 90 iterations:0.017986325975
delta after 100 iterations:0.0172515514365
delta after 110 iterations:0.0165922622597
delta after 120 iterations:0.0159805653848
delta after 130 iterations:0.0154014270289
delta after 140 iterations:0.0148203946705
delta after 150 iterations:0.0142621680648
delta after 160 iterations:0.0137040733542
delta after 170 iterations:0.0131268981937
delta after 180 iterations:0.0125645323366
delta after 190 iterations:0.0119991663579


C:\Users\anish\Anaconda3\lib\site-packages\ipykernel\__main__.py:5: RuntimeWarning: overflow encountered in exp


delta after 200 iterations:0.0114552487473
delta after 210 iterations:0.0109174146985
delta after 220 iterations:0.0104064945212
delta after 230 iterations:0.00993849367539
delta after 240 iterations:0.00952203529048
delta after 250 iterations:0.00915136819678
delta after 260 iterations:0.00881714016709
delta after 270 iterations:0.00848538211072
delta after 280 iterations:0.00815710401054
delta after 290 iterations:0.00783733191012
delta after 300 iterations:0.00754407436434
delta after 310 iterations:0.00727145979737
delta after 320 iterations:0.00701598562593
delta after 330 iterations:0.00676393089294
delta after 340 iterations:0.00654734792195
delta after 350 iterations:0.00635681627688
delta after 360 iterations:0.00618044916998
delta after 370 iterations:0.00601896185328
delta after 380 iterations:0.00585438429912
delta after 390 iterations:0.00570808228464
delta after 400 iterations:0.00558667640516
delta after 410 iterations:0.00547209342917
delta after 420 iterations:0.005346

In [326]:
#print (theta1.size)
#Result check
n=2000
l0 = sX[n]
print (training['sentence'][n+1])
print (training['subcategory'][n+1])
print (len(subcat), "subcategory",subcat)
# matrix multiplication of input and hidden layer
l1 = sigmoid(np.dot(l0, stheta1))
# output layer
l2 = sigmoid(np.dot(l1, stheta2))

print (l2)

How do you address a Chinese person ?

manner
47 subcategory ['cremat', 'manner', 'animal', 'exp', 'ind', 'gr', 'title', 'def', 'date', 'reason', 'event', 'state', 'desc', 'count', 'other', 'letter', 'religion', 'food', 'country', 'color', 'termeq', 'city', 'body', 'dismed', 'mount', 'money', 'product', 'period', 'substance', 'sport', 'plant', 'techmeth', 'volsize', 'instru', 'abb', 'speed', 'word', 'lang', 'perc', 'code', 'dist', 'temp', 'symbol', 'ord', 'veh', 'weight', 'currency']
[ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]


C:\Users\anish\Anaconda3\lib\site-packages\ipykernel\__main__.py:5: RuntimeWarning: overflow encountered in exp
